In [7]:
import numpy as np
import pandas as pd

In [8]:
np.loadtxt('..\\data\\ml-100k\\ua.test', skiprows=0, delimiter='\t').astype('int32')

array([[        1,        20,         4, 887431883],
       [        1,        33,         4, 878542699],
       [        1,        61,         4, 878542420],
       ...,
       [      943,       570,         1, 888640125],
       [      943,       808,         4, 888639868],
       [      943,      1067,         2, 875501756]])

In [9]:
def load_data(path, delimiter='\t'):
    train = np.loadtxt(path+'ua.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'ua.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:, 0]).size #num of users
    n_i = np.unique(total[:, 1]).size #num of items

    train_data = np.zeros((n_u, n_i), dtype='float32')
    test_data = np.zeros((n_u, n_i), dtype='float32')

    for i in range(train.shape[0]):
        train_data[train[i][0]-1][train[i][1]-1] = train[i][2]
    
    for i in range(test.shape[0]):
        test_data[test[i][0]-1][test[i][1]-1] = test[i][2]

    return train_data, test_data

In [10]:
train_data, test_data = load_data(path='..\\data\\ml-100k\\')

In [11]:
a = 1601
sum(train_data[:, a])/np.count_nonzero(train_data[:, a])

3.3333333333333335

In [12]:
def average_item_rating(train_data):
    list_average_item_rating = [sum(train_data[:, i])/np.count_nonzero(train_data[:, i]) for i in range(train_data.shape[1])]
    return list_average_item_rating

In [13]:
list_average_item_rating = average_item_rating(train_data=train_data)

C:\Users\Theba\AppData\Local\Temp\ipykernel_10856\2928453520.py:2: RuntimeWarning: invalid value encountered in scalar divide
  list_average_item_rating = [sum(train_data[:, i])/np.count_nonzero(train_data[:, i]) for i in range(train_data.shape[1])]


In [14]:
list_average_item_rating[1183]

2.5555555555555554

In [15]:
def get_item_features(item_path): 
   i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

   movies = pd.read_csv(item_path, sep='|', names=i_cols,encoding='latin-1')
   genres = movies[['unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
   genres = genres.to_numpy()

   return genres
    

In [16]:
item_featuers = get_item_features('..\\data\\ml-100k\\u.item')

In [17]:
item_featuers[0]

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [19]:
import transformers

In [20]:
import torch

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1) # 1 label for regression

# Sample input description
description = "This product is amazing! It exceeded all my expectations."

# Tokenize input description
inputs = tokenizer(description, return_tensors="pt", max_length=512, truncation=True)

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted rating
predicted_rating = outputs.logits.item()

print("Predicted Rating:", predicted_rating)


In [33]:
from torch.utils.data import DataLoader, Dataset

In [27]:
len(item_featuers)

1682

In [28]:
item_featuers

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
list_average_item_rating

[3.8596938775510203,
 3.1983471074380163,
 3.0588235294117645,
 3.5454545454545454,
 3.2911392405063293,
 3.4347826086956523,
 3.8034682080924855,
 4.025773195876289,
 3.876865671641791,
 3.8292682926829267,
 3.80184331797235,
 4.386454183266932,
 3.451219512195122,
 3.9565217391304346,
 3.780392156862745,
 3.1944444444444446,
 3.0941176470588236,
 2.8,
 3.9,
 3.3442622950819674,
 2.7721518987341773,
 4.164285714285715,
 4.114457831325301,
 3.4213836477987423,
 3.4302788844621515,
 3.417910447761194,
 3.1538461538461537,
 3.9518072289156625,
 2.6576576576576576,
 3.9722222222222223,
 3.6413793103448278,
 3.7564102564102564,
 3.460674157303371,
 2.7142857142857144,
 2.1818181818181817,
 2.0,
 2.25,
 3.0,
 3.269230769230769,
 2.8653846153846154,
 3.081081081081081,
 3.804195804195804,
 3.0,
 3.3552631578947367,
 4.0675675675675675,
 3.576923076923077,
 3.588709677419355,
 4.116504854368932,
 3.2911392405063293,
 4.365656565656566,
 3.473684210526316,
 3.7093023255813953,
 2.9590163934426

In [34]:
class CustomDataset(Dataset):
    def __init__(self, item_features: np.ndarray, lables: list) -> None:
        self.item_features = item_featuers
        self.lables = lables

    def __len__(self):
        return len(self.item_features)
    
    def __getitem__(self, idx) -> None:
        item = self.item_features[idx]
        lable = self.lables[idx]

        return item, lable

In [35]:
training_data = CustomDataset(item_features=item_featuers, lables=list_average_item_rating)

In [38]:
training_data_loader = DataLoader(training_data, batch_size=8, shuffle=True)